Activity 6-1
------------

The goal for this activity will be to compute some BCNF decompositions, using the tools from last lecture

First we'll load those tools, and some sample data:

In [1]:
from closure import compute_closure, display_side_by_side, print_setup

In [2]:
%load_ext sql
%sql sqlite://

/Users/tarabalakrishnan/.local/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/tarabalakrishnan/.local/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


'Connected: None@None'

In [3]:
%%sql DROP TABLE IF EXISTS T;
CREATE TABLE T(course VARCHAR, classroom INT, time INT);
INSERT INTO T VALUES ('CS 364', 132, 900);
INSERT INTO T VALUES ('CS 245', 140, 1000);
INSERT INTO T VALUES ('EE 101', 210, 900);

Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.


[]

### Exercise 1

First, let's decompose `T` into BCNF!  Explicitly go through the steps of the BCNF algorithm using the `compute_closure` function, then decompose the following table (i.e. by creating new SQL tables) into BCNF:

We've also made a function, `display_side_by_side`, for nicer display!

In [4]:
%sql SELECT * FROM T;

Done.


course,classroom,time
CS 364,132,900
CS 245,140,1000
EE 101,210,900


We are given the following FDs:

In [5]:
A = set(['course', 'classroom', 'time'])
F = [('course', 'classroom'), (set(['classroom', 'time']), 'course')]
print_setup(A, F)

Attributes = {classroom,course,time}
FDs = 	course -> classroom
	classroom,time -> course


**Q:** What real-world constraints do these FDs express?

Now, use the `compute_closure` function to help decompose this table to BCNF:

In [6]:
# We find an X s.t. X^+ != A and X^+ != X
X = set(['course'])
compute_closure(X, F)

{'classroom', 'course'}

Compose into two tables, $T_1$ and $T_2$:

In [12]:
%%sql
DROP TABLE IF EXISTS T1;
CREATE TABLE T1 AS SELECT DISTINCT * FROM (
        SELECT course, classroom FROM T
);

Done.
Done.


[]

In [13]:
%%sql
DROP TABLE IF EXISTS T2;
CREATE TABLE T2 AS SELECT DISTINCT * FROM (
        SELECT course, time FROM T
);

Done.
Done.


[]

Now run the below to display the decomposed tables side-by-side:

In [14]:
l = %sql SELECT * FROM T1;
r = %sql SELECT * FROM T2;
display_side_by_side(l,r)

Done.
Done.


course classroom CS 364 132 CS 245 140 EE 101 210 course time CS 364 900 CS 245 1000 EE 101 900

**Q:** Is this now in BCNF?

### Exercise 2

In the next section of lecture, we'll discuss a shortcoming of BCNF decompositions; let's see if we can get a glimpse of this now.

See if you can insert rows into $T_1$ and/or $T_2$ _which respect the local FDs that still hold_, such that **when $T_1$ and $T_2$ are now recomposed, the original FDs do not hold!**

In [15]:
%%sql
INSERT INTO T1 VALUES ('CS 145', 132);
INSERT INTO T2 VALUES ('CS 145', 900);

1 rows affected.
1 rows affected.


[]

Now, reconstruct and print the re-composed table using a SQL query:

In [16]:
%%sql
SELECT T1.course, T1.classroom, T2.time
FROM T1, T2
WHERE T1.course = T2.course;

Done.


course,classroom,time
CS 364,132,900
CS 245,140,1000
EE 101,210,900
CS 145,132,900


**Q:** What went wrong??  And how could we prevent this from occuring?